## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


## Business Understanding

King County has witnessed a vibrant real estate market. The housing market in this county is competitive and homeowners looking to buy or sell properties frequently seek guidence from local estate agencies. One of the key services they provide is advising homeowners on how renovations can potentially increase the value of their homes before listing them on the market. However, the agency faces the challenge of accurately quantifying the impact of various renovation features on the selling price of homes in the county. To address this challenge, the agency seeks to leverage data analysis and regression modeling to provide homeowners with informed recommendations on which renovations are likely to yield the highest returns.

### Business Problem: 
Having the right price for properties is important for real estate companies. It's not just about making sellers happy by getting them a good deal, but also about making buyers feel like they're getting a fair price. When a company consistently prices properties accurately, it builds trust in the community. People know they can rely on that company because they're transparent and fair.

The goal in this project is to analyze the relationship between various home features and sale price of the houses in KIng County. The aim is to provide insights and advice to stakeholders in the real estate industry about how they can improve their returns on investments by focusing on the features that have the most significant impact on the sale price of the houses.

### Objectives

1. To understand which factors determines the price of a home.

2. To understand how square feet living affect the value of a home.

3. To explore how condition affect the price of a home.

4. To explore which features will decrease and increase value of the house.

### Research Questions
1. Which factors determines the price of a home?

2. How square feet living affect the value of a home?

3. How condition affect the price of a home?

4. Which features will decrease and increase value of the house?

### Goal
This analysis will be guiding the real estate on the impact of various property attributes on their pricing structure. The analysis aims to offer a clear understanding of how property attributes collectively influence the estimated value of homes, enabling strategic decision for optimal return on investment.


### Data Understanding

We have been provided with a dataset with house sale prices in King County, Washington State, USA from 2014 to 2015 to use for this project.

A dataset has been provided and can be found in the kc_house_data.csv file in this repository.

The column names and descriptions as provided can be found in the column_names.md file in this repository. I have explained them here for convenience.

##### Column Names and Descriptions for Kings County Data Set

**id** : A notation for a house

**date**: Date house was sold

**price**: Price is prediction target

**bedrooms**: Number of bedrooms

**bathrooms**: Number of bathrooms

**sqft_living**: Square footage of the home

**sqft_lot**: Square footage of the lot

**floors** :Total floors (levels) in house

**waterfront** :House which has a view to a waterfront

**view**: Has been viewed

**condition** :How good the condition is overall

**grade**: overall grade given to the housing unit, based on King County grading system

**sqft_above** : Square footage of house apart from basement

**sqft_basement**: Square footage of the basement

**yr_built** : Built Year

**yr_renovated** : Year when house was renovated

**zipcode**: Zip code

**lat**: Latitude coordinate

**long**: Longitude coordinate

**sqft_living15** : Living room area in 2015(implies-- some renovations) This might or might not have affected the lotsize area

**sqft_lot15** : LotSize area in 2015(implies-- some renovations)
